In [1]:
import numpy as np
import pandas as pd
from scipy import sparse

In [2]:
# Initializing the sparse input and output matrix
D=pd.read_csv('../data/Semenov2017PaperData_matches_dota.csv',nrows=4600)
D=D.loc[(D['leavers']==0)&(D['skill']==1),:]
Out=sparse.csc_matrix(D['radiant_win'])
train_heroes=D.loc[:,'r1_hero_id':]
trainX=np.zeros((D.shape[0],2*113))
for i in range(D.shape[0]):
    trainX[i,train_heroes.iloc[i,0:5].values-1]=np.ones(5)
    trainX[i,train_heroes.iloc[i,5:10].values-1+113]=np.ones(5)
In=sparse.csr_matrix(trainX)

In [3]:
D

,match_id,skill,start_time,human_players,game_mode,lobby_type,first_blood_time,radiant_win,barracks_status_dire,barracks_status_radiant,...,r1_hero_id,r2_hero_id,r3_hero_id,r4_hero_id,r5_hero_id,d1_hero_id,d2_hero_id,d3_hero_id,d4_hero_id,d5_hero_id
4597,2144325432,1,1455198089,10,22,7,15,0,63,51,...,93,71,3,106,74,8,111,63,4,67


In [4]:
m=10000
j=0
for chunk in pd.read_csv('../data/Semenov2017PaperData_matches_dota.csv',chunksize=m):
    if j%50==0:
        print('Processing ',j+1,'st block of 10000 records')
    j+=1
        
    D=chunk.loc[(chunk['leavers']==0)&(chunk['skill']==1),:]
    n=D.shape[0]
    Out=sparse.hstack([Out,sparse.csc_matrix(D['radiant_win'])])
    train_heroes=D.loc[:,'r1_hero_id':]
    trainX=np.zeros((n,2*113))
    for i in range(D.shape[0]):
        trainX[i,train_heroes.iloc[i,0:5].values-1]=np.ones(5)
        trainX[i,train_heroes.iloc[i,5:10].values-1+113]=np.ones(5)
    In=sparse.vstack([In,sparse.csr_matrix(trainX)])

Processing  1 st block of 10000 records
Processing  51 st block of 10000 records
Processing  101 st block of 10000 records
Processing  151 st block of 10000 records
Processing  201 st block of 10000 records
Processing  251 st block of 10000 records
Processing  301 st block of 10000 records
Processing  351 st block of 10000 records
Processing  401 st block of 10000 records
Processing  451 st block of 10000 records
Processing  501 st block of 10000 records


In [5]:
In

<2469137x226 sparse matrix of type '<class 'numpy.float64'>'
	with 24691369 stored elements in Compressed Sparse Row format>

In [6]:
Out

<1x2469137 sparse matrix of type '<class 'numpy.int64'>'
	with 1333716 stored elements in Compressed Sparse Column format>

In [7]:
from sklearn.model_selection import train_test_split
tempX, TestX, tempY, TestY = train_test_split(In,Out.transpose(),test_size=0.25)
TrainX, ValidX, TrainY, ValidY = train_test_split(tempX,tempY,test_size=0.333)
print('TrainX size', TrainX.shape)
print('ValidX size', ValidX.shape)
print('TestX size', TestX.shape)
print('TrainY size', TrainY.shape)
print('ValidY size', ValidY.shape)
print('TestY size', TestY.shape)

TrainX size (1235185, 226)
ValidX size (616667, 226)
TestX size (617285, 226)
TrainY size (1235185, 1)
ValidY size (616667, 1)
TestY size (617285, 1)


In [8]:
sparse.save_npz('TrainInput_sparse_noleave_N.npz',TrainX)
sparse.save_npz('ValidInput_sparse_noleave_N.npz',ValidX)
sparse.save_npz('TestInput_sparse_noleave_N.npz',TestX)
sparse.save_npz('TrainOutput_sparse_noleave_N.npz',TrainY)
sparse.save_npz('ValidOutput_sparse_noleave_N.npz',ValidY)
sparse.save_npz('TestOutput_sparse_noleave_N.npz',TestY)

In [9]:
with open('AllSetSparseInOut_noleave_N.npz','wb') as handle:
    np.savez_compressed(handle,TrainX=TrainX,ValidX=ValidX,TestX=TestX,TrainY=TrainY,ValidY=ValidY,TestY=TestY)